In [1]:
finetuned_model_path = "/home/sagemaker-user/chatrd/scripts/uc_embedding_generation/notebooks/modern_bert_bone_lora2025-03-31_21-01-46"

In [2]:
import os
import re
from collections import defaultdict
import random
import json

def extract_keys(template):
    keys = re.findall(r"{(.*?)}", template)
    return keys


def get_dictionary_from_folders(directory_path: str):
    json_dict = {}

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            with open(file_path, "r") as file:
                json_content = json.load(file)

            key = os.path.splitext(filename)[0]

            json_dict[key] = json_content

    return json_dict
    
mappings = get_dictionary_from_folders("/home/sagemaker-user/chatrd/scripts/uc_embedding_generation/template_values")
template_values = get_dictionary_from_folders("/home/sagemaker-user/chatrd/scripts/uc_embedding_generation/templates")

def generate_questions(mappings, template_values, num_runs, K):
    questions = {}
    n_timer = defaultdict(lambda: 1, {key : K // len(template_values[key]) for key in template_values.keys()})
    for key, values in template_values.items():
        templates = values
        for template in templates:
            unique_questions = set()
            N = num_runs * n_timer[key]
            for _ in range(N):
                present_keys = extract_keys(template)
                all_pairs = {}
                for mp_key in present_keys:
                    all_pairs[mp_key] = random.choice(
                        mappings[mp_key],
                    )
                question = template.format(**all_pairs)
                unique_questions.add(question)
            if key in questions:
                questions[key].extend(list(unique_questions))
            else:
                questions[key] = list(unique_questions)
    return questions

In [3]:
import json
import logging
import os
from typing import Dict, List

import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

/home/sagemaker-user/.conda/envs/rapids-25.02/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sentence_transformers import SentenceTransformer
newmodel = SentenceTransformer(finetuned_model_path)

/home/sagemaker-user/.conda/envs/rapids-25.02/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from cuml.cluster import KMeans
import numpy as np
knn_q = generate_questions(mappings, template_values, 5, 10_000)
keys = list(knn_q.keys())
qs, cats = zip(*[(q, key) for key in keys for q in knn_q[key]])
le = LabelEncoder()
y = le.fit_transform(cats)
tqx, TqX, ty, TY = train_test_split(qs, y, shuffle=True)

In [6]:
len(qs)

244485

In [7]:
from scipy.stats import mode
from sklearn.neighbors import KNeighborsClassifier

In [9]:
# # Given cluster predictions for the training data with 100% homogeneous clusters we want to map the cluster categories to the training categories and then use this mapping to compress the training data
dim = 64
model = newmodel
tx, TX = model.encode(tqx), model.encode(TqX)
tldx = tx[:, :dim]

In [10]:
dim = 64
tldx = tx[:, :dim]
km = KMeans(n_clusters = 500)
km.fit(tldx)
kyh = km.predict(tldx)
lyh = np.asarray([mode(ty[kyh == k])[0] if (kyh == k).sum() != 0 else -1 for k in range(km.n_clusters)]) 
knn_ld_km = KNeighborsClassifier(n_neighbors = 1, metric="euclidean")
knn_ld_km.fit(km.cluster_centers_, lyh)
knn_ld_km.score(TX[:, :dim], TY)

/home/sagemaker-user/.conda/envs/rapids-25.02/lib/python3.12/site-packages/cuml/internals/api_decorators.py:195: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  return func(*args, **kwargs)
/home/sagemaker-user/.conda/envs/rapids-25.02/lib/python3.12/site-packages/cuml/internals/api_decorators.py:193: FutureWarning: The default value of `n_init` will change from 1 to 'auto' in 25.04. Set the value of `n_init` explicitly to suppress this warning.
  ret = func(*args, **kwargs)


0.9999018356729165

In [11]:
np.mean([np.mean(lyh[k] == ty[kyh == k]) for k in range(km.n_clusters)]) # average majority class percentage of clusters, should be above 99% to continue.

np.float64(0.999268817204301)

In [12]:
py = knn_ld_km.predict(TX[:, :dim])
pyl = le.inverse_transform(py)

In [13]:
import pandas as pd
from IPython.display import Markdown
Markdown(pd.DataFrame(list(zip(np.asarray(TqX)[TY != py], pyl[TY != py])), columns = ["Question", "Predicted Category"]).to_markdown())

|    | Question                                                                                                                       | Predicted Category   |
|---:|:-------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | What is the meaning or definition of 'rising star'?                                                                            | definition           |
|  1 | Can you provide a list of potential rising stars in the industry?                                                              | query                |
|  2 | Show me the Volkswagen rating history                                                                                          | general              |
|  3 | Can you provide the arvada, co lease appropriation Days Sales Outstanding (DSO) data for 2024 and 2024 broken down by quarter? | general              |
|  4 | How does the latest research on target corporation compare to the latest research on Facebook, Inc. (Meta Platforms)?          | general              |
|  5 | Summary/updates about Shawan LLC 2001-A Senior PMIs                                                                            | scores&modifiers     |

In [14]:
kmccx = km.cluster_centers_

In [15]:
lyhc = [str(le.classes_[yi]) if yi != -1 else 'general' for yi in lyh]

In [16]:
len(lyhc)

500

In [17]:
import pandas as pd
pd.Series(lyhc).values.astype('U')

array(['criteria', 'deals_tranche', 'outlook', 'esg', 'financials',
       'scores&modifiers', 'ratings', 'general', 'peers', 'research',
       'sNw', 'definition', 'query', 'rating_action', 'credit_memo',
       'securities', 'criteria', 'general', 'deals_tranche', 'general',
       'research', 'general', 'query', 'general', 'financials', 'ratings',
       'criteria', 'general', 'criteria', 'general', 'outlook', 'outlook',
       'general', 'general', 'definition', 'general', 'financials',
       'research', 'peers', 'esg', 'criteria', 'general',
       'scores&modifiers', 'deals_tranche', 'general', 'sNw', 'sNw',
       'ratings', 'criteria', 'general', 'research', 'general', 'general',
       'outlook', 'general', 'general', 'outlook', 'deals_tranche',
       'research', 'criteria', 'definition', 'financials', 'criteria',
       'research', 'query', 'general', 'scores&modifiers', 'general',
       'general', 'ratings', 'general', 'deals_tranche', 'criteria',
       'ratings', 'gene

In [18]:
np.savez('use_case_embeddings', embeddings = kmccx, categories = pd.Series(lyhc).values.astype('U'))

In [19]:
kmccx.shape, len(pd.Series(lyhc).values.astype('U'))

((500, 64), 500)

In [20]:
# kmx, kmy, s = get_knn(mpnetmodel, 64, 300)
current_knn = KNeighborsClassifier(n_neighbors = 1)
embeddings = np.load("/home/sagemaker-user/chatrd/chatrd/engine/data/use_case_embeddings.npz")
current_knn.fit(embeddings['embeddings'], embeddings['categories'])

KNeighborsClassifier(n_neighbors=1)

In [21]:
# kmx, kmy, s = get_knn(mpnetmodel, 64, 300)
new_knn = KNeighborsClassifier(n_neighbors = 1)
# embeddings = np.load("/home/sagemaker-user/chatrd/chatrd/engine/data/use_case_embeddings.npz")
new_knn.fit(kmccx, lyhc)

KNeighborsClassifier(n_neighbors=1)

In [22]:
def make_pmodel(knn, model, dim=dim):
    return lambda x: str(knn.predict(model.encode(x).reshape(1, -1)[:, :dim])[0])
pred_model = make_pmodel(new_knn, newmodel, 64)

Test questions for the model

In [23]:
%%writefile temp
What is the outlook of South Korea's economy?
What is the outlook of China's economy?
Outlook for Austria
provide a good overview of the outlook for colombia
What is the macro economic outlook for the US?
What is the current outlook for Boeing?
outlook for AIG Inc.
What is the outlook of American Airlines?
What is the economic outlook for Cambodia?
What is the outlook for Warner Music Group?
Verizon outlook
what is the economic outlook for Disney
Compare the outlook of JP Morgan to HSBC
What is the outlook for Goldman Sachs compared to Morgan Stanley?
what is the outlook for tencent?
outlook for Malaysia in the next 1-2 years
Outlook for Microsoft
please give me the outlook for TSMC
what could impact the outlook for UBS
What is the outlook of delhi international airport?
outlook for City of New York
outlook for Virginia Port Authority
What is the outlook for Heathrow Funding Limited
What was the latest outlook for Yale Univ, CT General Obligation?
what could cause an upgrade for Johnson & Johnson
what would trigger an upgrade of midea group
Compare the upgrade of Microsoft to Google
what can cause credit ratings downgrades for samsung electronics?
what can cause ratings downgrade for TSMC?
What would cause a downgrade for Chevron?
What are the potential triggers for a downgrade in the credit ratings of Tesla
Outlook for oil & gas sector
What is the outlook for Telecom sector?
What is the outlook for Indonesian banking sector?
What is the outlook for Chinese real estate sector?
What is the outlook on Healthcare
what is the global macro economic outlook
what is the United Kingdom's Utility sector outlook?
what is the economic outlook in APAC for the motor vehicles sector?
What is the outlook for auto sector in the US
Could you please share with me recent outlook on the real estate sector in Europe?
Please find me a recent outlook on life sciences industry?
what is the outlook for energy industry
Outlook for artificial intelligence companies
What is the outlook of the banking industry
What is outlook for office REITs?
what is the outlook on the sports sector
industries with a positiv outlook in austria
How is the luxury sector credit outlook look for 2025?
what is the outlook for the indonesian coal market
What is the current banking sector outlook for Taiwan?
what is the outlook for southeast asia
What is the outlook of North America transportation  industry?
What are Electric vehicle industry outlook
what is the outlook for the Canadian forestry industry
How does the healthcare sector's credit outlook look?
What is the current economic outlook for South Korea?
What is the current economic outlook for China?
What is the current economic or financial outlook for Austria?
Can you provide a good overview of the economic and financial outlook for Colombia?
What is the current macroeconomic outlook for the United States?
What is the current outlook for Boeing?
What is the current outlook for AIG Inc.?
What is the current outlook for American Airlines?
What is the current economic outlook for Cambodia?
What is the current outlook for Warner Music Group?
What is the outlook for Verizon?
What is the economic outlook for Disney?
Can you compare the outlook of JP Morgan to HSBC?
What is the outlook for Goldman Sachs compared to the outlook for Morgan Stanley?
What is the current outlook for Tencent?
What is the economic and financial outlook for Malaysia in the next 1-2 years?
What is the outlook for Microsoft?
What is the outlook for TSMC?
What factors could impact the outlook for UBS?
What is the current outlook for the Delhi International Airport?
What is the current outlook for the City of New York?
What is the current outlook for the Virginia Port Authority?
What is the current outlook for Heathrow Funding Limited?
What is the latest outlook for the Yale Univ, CT General Obligation?
What factors could lead to an upgrade in Johnson & Johnson's credit rating?
What factors or events would trigger an upgrade of Midea Group's credit rating?
Compare the upgrade of Microsoft to Google
What factors can lead to credit rating downgrades for Samsung Electronics?
What factors can lead to a ratings downgrade for TSMC (Taiwan Semiconductor Manufacturing Company)?
What factors could lead to a downgrade in Chevron's credit rating?
What are the potential triggers for a downgrade in Tesla's credit ratings?
What is the outlook for the oil and gas sector?
What is the current outlook for the Telecom sector?
What is the current outlook for the Indonesian banking sector?
What is the current outlook for the Chinese real estate sector?
What is the current outlook for the healthcare industry?
What is the global macroeconomic outlook?
What is the current outlook for the United Kingdom's Utility sector?
What is the economic outlook in the Asia-Pacific (APAC) region for the motor vehicles sector?
What is the current outlook for the auto sector in the United States?
What is the recent outlook on the real estate sector in Europe?
What is the recent outlook for the life sciences industry?
What is the outlook for the energy industry?
What is the outlook for artificial intelligence companies?
What is the current outlook for the banking industry?
What is the outlook for office REITs?
What is the outlook for the sports sector?
What industries in Austria have a positive outlook?
What is the credit outlook for the luxury sector in 2025?
What is the outlook for the Indonesian coal market?
What is the current outlook for the banking sector in Taiwan?
What is the economic or financial outlook for Southeast Asia?
What is the current outlook for the North America transportation industry?
What is the outlook for the electric vehicle industry?
What is the outlook for the Canadian forestry industry?
What is the current credit outlook for the healthcare sector?
what is the latest research for IBM
What's the latest research on Coca-Cola Company
What's the latest research on <(NYSE:C) Citigroup Inc.>
What is the latest research on The Coca-Cola Company?
What is the latest research on Citigroup Inc. (NYSE:C)?

Writing temp


In [24]:
with open("temp", "r") as f:
    lines = f.read().strip().split('\n')
lines
qs = lines

In [25]:
[(q, c) for (c, q) in zip(map(pred_model, qs), qs)]

[("What is the outlook of South Korea's economy?", 'outlook'),
 ("What is the outlook of China's economy?", 'outlook'),
 ('Outlook for Austria', 'general'),
 ('provide a good overview of the outlook for colombia', 'outlook'),
 ('What is the macro economic outlook for the US?', 'outlook'),
 ('What is the current outlook for Boeing?', 'outlook'),
 ('outlook for AIG Inc.', 'general'),
 ('What is the outlook of American Airlines?', 'outlook'),
 ('What is the economic outlook for Cambodia?', 'outlook'),
 ('What is the outlook for Warner Music Group?', 'outlook'),
 ('Verizon outlook', 'outlook'),
 ('what is the economic outlook for Disney', 'outlook'),
 ('Compare the outlook of JP Morgan to HSBC', 'outlook'),
 ('What is the outlook for Goldman Sachs compared to Morgan Stanley?',
  'outlook'),
 ('what is the outlook for tencent?', 'outlook'),
 ('outlook for Malaysia in the next 1-2 years', 'general'),
 ('Outlook for Microsoft', 'general'),
 ('please give me the outlook for TSMC', 'outlook'),
